In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [ ]:
# create keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS datahungry
  WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 3
  };
""").one()

In [ ]:
# use keyspace
session.set_keyspace('datahungry')

In [ ]:
# create table
session.execute("""
CREATE TABLE IF NOT EXISTS registered_profile_dev(
    job text,
    company text,
    ssn text,
    residence text,
    current_location tuple<float, float>,
    blood_group text,
    website list<text>,
    username text,
    name text,
    sex text,
    address text,
    mail text,
    birthdate date,
    PRIMARY KEY (ssn)
   );
""").one()

In [ ]:
# please, run faker_to_kafka.py to produce before consuming

In [ ]:
# consume from kafka to insert canssandra using json
from kafka import KafkaConsumer

cassandra_tablename = "registered_profile"
kafka_topicname = "registered_profile"

bootstrap_servers = ['localhost:9092']
consumer = KafkaConsumer(kafka_topicname,
                         bootstrap_servers=bootstrap_servers,
                         auto_offset_reset='latest')
for message in consumer:
    data = message.value.decode('utf-8').replace("\'", "")
    query = f"INSERT INTO {cassandra_tablename} JSON '{data}';"
    session.execute(query)

In [ ]:
# get data
r = session.execute("SELECT * FROM registered_profile")
df = pd.DataFrame(r)
df